In [20]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Bidirectional,LSTM,Dense,Embedding
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [21]:
with open("./pure_cc_clues.pickle",'rb') as f:
    dfs = pickle.load(f)

In [22]:
def split_dfs(dfs,shuffle=False):
    train_test = [train_test_split(df,shuffle=shuffle,test_size=0.2) for df in dfs ]
    train = [el[0] for el in train_test]
    test = [el[1] for el in train_test]
    return train,test

In [23]:
inpt,test = split_dfs(dfs)

In [24]:
train,val = split_dfs(inpt,shuffle=True)

In [25]:
train = pd.concat(train)
val = pd.concat(val)
test = pd.concat(test)

In [26]:
train = train.drop(['is_charade','is_lit'],axis=1)
val = val.drop(['is_charade','is_lit'],axis=1)
test = test.drop(['is_charade','is_lit'],axis=1)

In [27]:
train_x = train.clue
val_x = val.clue
test_x = test.clue

In [28]:
train_y = train[train.columns[4:]]
val_y = val[train.columns[4:]]
test_y = test[test.columns[4:]]

In [29]:
upsampled_train = train

In [40]:
categories = np.argmax(train_y.values*1,axis=1)

In [42]:
upsampled_train['category'] = categories

In [45]:
max_size = upsampled_train.groupby('category').count().max()[0]

In [46]:
lst = [upsampled_train]
for class_index, group in upsampled_train.groupby('category'):
    sample = group.sample(max_size-len(group), replace=True, )
    lst.append(sample)
upsampled_train = pd.concat(lst)

In [177]:
tokenizer = Tokenizer(filters="#$%&()*+/:;<=>@[\]^_`{|}~")
tokenizer.fit_on_texts(pd.concat([train,val,test]).clue.tolist())

In [178]:
train_x = pad_sequences(tokenizer.texts_to_sequences(train_x),15)
val_x = pad_sequences(tokenizer.texts_to_sequences(val_x),15)
test_x = pad_sequences(tokenizer.texts_to_sequences(test_x),15)

In [182]:
model = keras.Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1,output_dim=100,input_length=15))
model.add(Bidirectional(LSTM(100,dropout=0.5)))
model.add(Dense(12,activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

In [185]:
model.fit(train_x,train_y,validation_data=(val_x,val_y),epochs=10,batch_size=1024)

Train on 30063 samples, validate on 7524 samples
Epoch 1/10
30063/30063 [==============================] - 9s 302us/step - loss: 0.2457 - categorical_accuracy: 0.7237 - val_loss: 1.4324 - val_categorical_accuracy: 0.4381
Epoch 2/10
30063/30063 [==============================] - 9s 305us/step - loss: 0.1899 - categorical_accuracy: 0.7391 - val_loss: 1.4707 - val_categorical_accuracy: 0.4337
Epoch 3/10
30063/30063 [==============================] - 9s 309us/step - loss: 0.1565 - categorical_accuracy: 0.7491 - val_loss: 1.5529 - val_categorical_accuracy: 0.4215
Epoch 4/10
30063/30063 [==============================] - 9s 307us/step - loss: 0.1332 - categorical_accuracy: 0.7564 - val_loss: 1.5849 - val_categorical_accuracy: 0.4365
Epoch 5/10
30063/30063 [==============================] - 9s 308us/step - loss: 0.1109 - categorical_accuracy: 0.7624 - val_loss: 1.7225 - val_categorical_accuracy: 0.4329
Epoch 6/10
30063/30063 [==============================] - 9s 308us/step - loss: 0.0955 - ca

In [69]:
len(test)/(len(train)+len(test))

0.2001234278904471

In [77]:
!open .